In [8]:
pip install jmespath parsel unidecode pyodbc beautifulsoup4

   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   ------------------------------ --------- 112.6/147.9 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 147.9/147.9 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [147]:
import json
import requests
import jmespath
from parsel import Selector
import os
import bs4
import unidecode
import re
from ExtractData import *
from bs4 import BeautifulSoup
from tqdm import tqdm


In [24]:
f = open('mapComic.json')
ListComic = json.load(f)

In [ ]:
for idx,oneComic in  enumerate(ListComic["comic"]):
    flag = os.path.isfile(f"ComicData/{oneComic['urlComic']}.json")
    if(not flag) :
        # continue
        print(idx,oneComic['urlComic'])
    # f = open(f"ComicData/{oneComic['urlComic']}.json")
    # data = json.load(f)
    # if (not data['URL']):
    #     print(idx,oneComic['urlComic'])
    #     f.close()
    #     os.remove (f"ComicData/{oneComic['urlComic']}.json")

In [14]:
import requests


## Collect all comic from nettruyen

In [ ]:
def ExtractNetruyenComic(page = 1):
  url = f"https://nhattruyenbing.com/the-loai?status=-1&sort=10&page={page}"
  headers = {
      "Accept":"*/*",
      "User-Agent": "Thunder Client (https://www.thunderclient.com)"
  }
  response = requests.get(url, headers=headers)
  html_content = response.text
  soup = BeautifulSoup(html_content, 'html.parser')
  elements = soup.find_all('div',attrs={"class": ["item"]})

  _list_comic = []
  for element in elements:
    Comic = {}
    link = element.find('div',attrs={"class": ["image"]}).a.get('href')
    title = element.find('div',attrs={"class": ["image"]}).a.get('title')
    lastUpdate = element.find('ul',attrs={"class": ["comic-item"]}).li.i.get_text()
    lastChapter = element.find('ul',attrs={"class": ["comic-item"]}).li.a.get_text()
    infos = element.find('div',attrs={"class": ["image"]}).div.span
    slug = CreateSlug(title)
    Comic['title'] = title
    Comic['url'] = link
    Comic['slug'] = slug
    Comic['last_update'] = convert_string_time_to_datetime(lastUpdate).strftime("%Y-%m-%d %H:%M:%S")
    Comic['last_chapter'] = lastChapter
    Comic['info'] = infos.get_text().strip()
    _list_comic.append(Comic)
    
  return _list_comic
    # break
all_comic = {"comic":[]}

for i in range(1, 600):
  print("Page ",i)
  data_comic = ExtractNetruyenComic(i)
  if(data_comic == []): break
  all_comic["comic"].extend(data_comic)
SaveJSON("all_comic_netruyen",all_comic)


In [ ]:
import time
def fetch_detail_comic_from_netruyen(comics:dict, _from, _to):
  for i in range(_from, _to):
    comic = comics[i]
    url = comic['url']
    headers = {
        "Accept":"*/*",
        "User-Agent": "Thunder Client (https://www.thunderclient.com)"
    }
    response = requests.get(url, headers=headers)
    if(response.status_code != 200): continue
    pattern = r'<article\b[^>]*>(.*?)</article>'
    single_comic = {}
    html_content = re.findall(pattern, response.text, re.DOTALL)[0]
    if(not html_content): continue
    soup = BeautifulSoup(html_content, 'html.parser')

    ul_element = soup.find('ul',attrs={"class": ["list-info"]}).find_all("p",attrs={"class": "col-xs-8"})
    single_comic["title"] = soup.h1.get_text()
    date_format = "[Cập nhật lúc: %H:%M %d/%m/%Y]"
    single_comic["update_at"] =datetime.strptime(soup.time.get_text().strip(), date_format).strftime("%Y-%m-%d %H:%M:%S")
    single_comic["author"] = ul_element[0].get_text()
    single_comic["status"] = ul_element[1].get_text()
    single_comic["view"] = ul_element[3].get_text()
    single_comic["genre"] = ul_element[2].get_text().split(" - ")
    single_comic["rating_value"] = soup.find("span",attrs={"itemprop": "ratingValue"}).get_text()
    single_comic["rating_count"] = soup.find("span",attrs={"itemprop": "ratingCount"}).get_text()           
    single_comic["follow_count"] = soup.find("div",attrs={"class": "follow"}).b.get_text()
    single_comic["chapters"] = []
    list_chap_elements = soup.find("div",class_="list-chapter").find_all("li",class_="row")
    for li in list_chap_elements:
      chapter ={
          "chater_name": li.a.get_text(),
          "update_at": convert_string_time_to_datetime(li.find("div",class_ ="col-xs-4").get_text()).strftime("%Y-%m-%d %H:%M:%S"),
          "view": li.find("div",class_ ="col-xs-3").get_text(),
        }
      single_comic["chapters"].append(chapter)
    SaveJSON(f"NettruyenData/{comic['slug']}",single_comic)


fetch_detail_comic_from_netruyen(all_comic["comic"], )


In [237]:
len(all_comic["comic"])

21952

In [ ]:
# import threading
# for i in range(10000,22000,1000):
#     th1 = threading.Thread(target=fetch_detail_comic_from_netruyen, args=(all_comic["comic"],i, i+1000))
#     th1.start()

In [3]:
import pyodbc 
server = 'DESKTOP-33SUFGJ\\SQLEXPRESS'
database = 'ComicApp'
username = 'your_username'
password = 'your_password'
connectionString = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};'
conn = pyodbc.connect(connectionString)
cursor = conn.cursor()

In [75]:
# connectionString = 'C:/Users/ACER/Desktop/ComicApp/SQL/ComicDB.db'


In [6]:
import glob

listPath = glob.glob("NettruyenData/*")
len(listPath)

21935

#### Insert comic to database

In [ ]:

def convertStatus(status):
  return 0 if(status == "Đang tiến hành") else 1
def convert_comic_sql_value(datajson):
  title = datajson['title']
  title = title.replace("'", "")
  value = f"(N'{title}','{CreateSlug(datajson['title'])}',{convertStatus(datajson['status'])},'{datajson['update_at']}','{datajson['update_at']}',10)"
  return value

def insert_1000_row_comic(_from, _to):
  
  SQL_QUERY = """
    INSERT INTO COMIC (Title,URL,Status,CreateAt,UpdateAt,Rating) VALUES
    {0}
  """
  ListValue = []
  for path in listPath[_from:_to]:
    f = open(path)
    datajson = json.load(f)
    value = convert_comic_sql_value(datajson)
    ListValue.append(value)

  values = ",".join(ListValue)
  print("Start:",_from,"-", _to)
  SQL_QUERY =SQL_QUERY.format(values)
  cursor.execute(SQL_QUERY)
  conn.commit()
for i in range(0,len(listPath),1000):
  insert_1000_row_comic(i,i+1000)

#### Query all Genre

In [ ]:
SQL_QUERY_GENRE = """
  Select * from GENRE
"""
cursor.execute(SQL_QUERY_GENRE)
allGenre = cursor.fetchall()
dictGenre = {}
for genre in allGenre:
  slug = CreateSlug(genre[1])
  dictGenre[slug] = genre[0]
dictGenre

#### Insert genre into comic

In [115]:
def FindComicID(cursor,urlComic):
  #sql query
  SQL_QUERY = f"""
  Select * from COMIC where URL = '{urlComic}'
  """
  cursor.execute(SQL_QUERY)
  comic = cursor.fetchone()
  if(comic):
    return comic[0]
  return None

In [146]:
SQL_QUERY = """
 INSERT INTO COMIC_GENRE (comicid,genreid) VALUES
  {0}
"""
genre_not_found = {}
for path in tqdm(listPath):
  f = open(path)
  datajson = json.load(f)
  comicid = FindComicID(cursor,CreateSlug(datajson['title']))
  if(comicid == None): 
    print("can't find comicid " +datajson['title'])
    continue
  myset = set(datajson['genre'])
  values = []
  for genre in myset:
    slug = CreateSlug(genre)
    if(slug not in dictGenre): 
      genre_not_found[genre] = 1
      continue
    value = f'({comicid},{dictGenre[slug]})'
    values.append(value)
  if(values):
    query = SQL_QUERY.format(",\n".join(values))
    cursor.execute(query)
    conn.commit()

#### Insert chapter to comic

In [ ]:

def convertStatus(status):
  return 0 if(status == "Đang tiến hành") else 1
def convert_comic_sql_value(datajson):
  title = datajson['title']
  title = title.replace("'", "")
  value = f"(N'{title}','{CreateSlug(datajson['title'])}',{convertStatus(datajson['status'])},'{datajson['update_at']}','{datajson['update_at']}',10)"
  return value

def insert_n_chapter_to_comic(comicid,_from, _to):
  
  SQL_QUERY = """
    INSERT INTO COMIC (Title,URL,Status,CreateAt,UpdateAt,Rating) VALUES
    {0}
  """
  ListValue = []
  for path in listPath[_from:_to]:
    f = open(path)
    datajson = json.load(f)
    value = convert_comic_sql_value(datajson)
    ListValue.append(value)

  values = ",".join(ListValue)
  print("Start:",_from,"-", _to)
  SQL_QUERY =SQL_QUERY.format(values)
  cursor.execute(SQL_QUERY)
  conn.commit()
for i in range(0,len(listPath),1000):
  insert_1000_row_comic(i,i+1000)

In [175]:
datajson = json.load(open("NettruyenData/vo-luyen-dinh-phong.json"))
datajson

{'title': 'Võ Luyện Đỉnh Phong',
 'update_at': '2024-04-04 10:01:00',
 'author': 'Đang cập nhật',
 'status': 'Đang tiến hành',
 'view': 'N/A',
 'genre': ['Action',
  'Fantasy',
  'Manhua',
  'Shounen',
  'Supernatural',
  'Truyện Màu'],
 'rating_value': '3.5',
 'rating_count': '1145670',
 'follow_count': '191.848',
 'chapters': [{'chater_name': 'Chapter 3726',
   'update_at': '7 ngày trước',
   'view': 'N/A'},
  {'chater_name': 'Chapter 3725', 'update_at': '8 ngày trước', 'view': 'N/A'},
  {'chater_name': 'Chapter 3724', 'update_at': '9 ngày trước', 'view': 'N/A'},
  {'chater_name': 'Chapter 3723', 'update_at': '10 ngày trước', 'view': 'N/A'},
  {'chater_name': 'Chapter 3722', 'update_at': '11 ngày trước', 'view': 'N/A'},
  {'chater_name': 'Chapter 3721', 'update_at': '12 ngày trước', 'view': 'N/A'},
  {'chater_name': 'Chapter 3720', 'update_at': '13 ngày trước', 'view': 'N/A'},
  {'chater_name': 'Chapter 3719', 'update_at': '14 ngày trước', 'view': 'N/A'},
  {'chater_name': 'Chapter 3

In [ ]:
datajson['chapters']

In [213]:
SQL_QUERY = """
INSERT INTO CHAPTER (comicid, Title, URL,ChapterNumber,ViewCount,UpdateAt) VALUES
{0}
"""
comicid = FindComicID(cursor,CreateSlug(datajson['title']))
chapters = list(reversed(datajson['chapters']))
length = len(chapters)
loop = (length-1)//1000 +1
for i in range(loop):
    slice_chapter = chapters[i*1000:i*1000+1000]
    values = []
    for j,chapter in   enumerate(slice_chapter):
        time_update = convert_string_time_to_datetime(chapter['update_at']).strftime("%Y-%m-%d %H:%M:%S")
        view = chapter['view'].replace(".","")
        if(chapter['view'] == "N/A"): view = "0"
        chapter_name = chapter["chater_name"]
        value = f"({comicid},N'{chapter_name}','{CreateSlug(chapter_name)}',{i*1000+j+1},{view},'{time_update}')"
        values.append(value)
        
    query = SQL_QUERY.format(",\n".join(values))
    cursor.execute(query)
    conn.commit()
    


INSERT INTO CHAPTER (comicid, Title, ChapterNumber,ViewCount,UpdateAt) VALUES
(49402,N'Chapter 1',1,0,'2018-01-02 00:00:00'),
(49402,N'Chapter 2',2,0,'2018-01-03 00:00:00'),
(49402,N'Chapter 3',3,0,'2018-01-14 00:00:00'),
(49402,N'Chapter 4',4,0,'2018-01-16 00:00:00'),
(49402,N'Chapter 5',5,0,'2018-01-23 00:00:00'),
(49402,N'Chapter 6',6,0,'2018-01-27 00:00:00'),
(49402,N'Chapter 7',7,0,'2018-01-29 00:00:00'),
(49402,N'Chapter 8',8,0,'2018-01-30 00:00:00'),
(49402,N'Chapter 9',9,0,'2018-02-03 00:00:00'),
(49402,N'Chapter 10',10,0,'2018-02-04 00:00:00'),
(49402,N'Chapter 11',11,0,'2018-02-11 00:00:00'),
(49402,N'Chapter 12',12,0,'2018-02-17 00:00:00'),
(49402,N'Chapter 13',13,0,'2018-02-25 00:00:00'),
(49402,N'Chapter 14',14,0,'2018-02-26 00:00:00'),
(49402,N'Chapter 15',15,0,'2018-03-11 00:00:00'),
(49402,N'Chapter 16',16,0,'2018-03-12 00:00:00'),
(49402,N'Chapter 17',17,0,'2018-03-13 00:00:00'),
(49402,N'Chapter 18',18,0,'2018-03-17 00:00:00'),
(49402,N'Chapter 19',19,0,'2018-03-28 0

In [211]:
   conn.commit()